# Tutorial: The Basic Tools of Private Deep Learning

Welcome to PySyft's introductory tutorial for privacy preserving, decentralized deep learning. This series of notebooks is a step-by-step guide for you to get to know the new tools and techniques required for doing deep learning on secret/private data/models without centralizing them under one authority.

**Scope:** Note that we'll not just be talking about how to decentralized / encrypt data, but we'll be addressing how PySyft can be used to help decentralize the entire ecosystem around data, even including the Databases where data is stored and queried, and the neural models which are used to extract information from data. As new extensions to PySyft are created, these notebooks will be extended with new tutorials to explain the new functionality.

Authors:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)

## Outline:

- Part 1: The Basic Tools of Private Deep Learning


## Why Take This Tutorial?

**1) A Competitive Career Advantage** - For the past 20 years, the digital revolution has made data more and more accessible in ever larger quantities as analog processes have become digitized. However, with new regulation such as [GDPR](https://eugdpr.org/), enterprises are under pressure to have less freedom with how they use - and more importantly how they analyze - personal information. **Bottom Line:** Data Scientists aren't going to have access to as much data with "old school" tools, but by learning the tools of Private Deep Learning, YOU can be ahead of this curve and have a competitive advantage in your career. 

**2) Entrepreneurial Opportunities** - There are a whole host of problems in society that Deep Learning can solve, but many of the most important haven't been explored because it would require access to incredibly sensitive information about people (consider using Deep Learning to help people with mental or relationship issues!). Thus, learning Private Deep Learning unlocks a whole host of new startup opportunities for you which were not previously available to others without these toolsets.

**3) Social Good** - Deep Learning can be used to solve a wide variety of problems in the real world, but Deep Learning on *personal information* is Deep Learning about people, *for people*. Learning how to do Deep Learning on data you don't own represents more than a career or entrepreneurial opportunity, it is the opportunity to help solve some of the most personal and important problems in people's lives - and to do it at scale.

## How do I get extra credit?

- Star PySyft on GitHub! - [https://github.com/OpenMined/PySyft](https://github.com/OpenMined/PySyft)
- Make a Youtube video teaching this notebook!


... ok ... let's do this!

# Part -1: Prerequisites

- Know PyTorch - if not then take the http://fast.ai course and come back
- Read the PySyft Framework Paper https://arxiv.org/pdf/1811.04017.pdf! This will give you a thorough background on how PySyft is constructed which will help things make more sense.

# Part 0: Setup

To begin, you'll need to make sure you have the right things installed. To do so, head on over to PySyft's readme and follow the setup instructions. TLDR for most folks is.

- Install Python 3.5 or higher
- Install PyTorch 1.0
- Clone PySyft (git clone https://github.com/OpenMined/PySyft.git)
- cd PySyft
- pip install -r requirements.txt
- python setup.py install
- python setup.py test

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a GitHub Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [3]:

!pip install syft

     |████████████████████████████████| 184kB 2.8MB/s 
     |████████████████████████████████| 71kB 25.2MB/s 
     |████████████████████████████████| 450kB 50.3MB/s 
     |████████████████████████████████| 1.4MB 47.8MB/s 
     |████████████████████████████████| 204kB 46.3MB/s 
     |████████████████████████████████| 389kB 41.1MB/s 
     |████████████████████████████████| 276kB 46.0MB/s 
     |████████████████████████████████| 51kB 20.7MB/s 
     |████████████████████████████████| 122kB 53.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
# Run this cell to see if things work
import syft as sy
from syft.frameworks.torch.pointers import PointerTensor
from syft.frameworks.torch.tensors.decorators import LoggingTensor
import sys
import torch
hook = sy.TorchHook(torch)
from torch.nn import Parameter
import torch.nn as nn
import torch.nn.functional as F

torch.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

If this cell executed, then you're off to the races! Let's do this!

# Part 1: The Basic Tools of Private, Decentralized Data Science

So - the first question you may be wondering is - How in the world do we train a model on data we don't have access to? 

Well, the answer is surprisingly simple. If you're used to working in PyTorch, then you're used to working with torch.Tensor objects like these!

In [5]:
x = torch.tensor([1,2,3,4,5])
y = x + x
print(y)

tensor([ 2,  4,  6,  8, 10])


Obviously, using these super fancy (and powerful!) tensors is important, but also requires you to have the data on your local machine. This is where our journey begins. 

# Section 1.1 - Sending Tensors to Bob's Machine

Whereas normally we would perform data science / deep learning on the machine which holds the data, now we want to perform this kind of computation on some **other** machine. More specifically, we can no longer simply assume that the data is on our local machine.

Thus, instead of using Torch tensors, we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [0]:
bob = sy.VirtualWorker(hook, id="bob")

Let's say Bob's machine is on another planet - perhaps on Mars! But, at the moment the machine is empty. Let's create some data so that we can send it to Bob and learn about pointers!

In [0]:
x = torch.tensor([1,2,3,4,5])
y = torch.tensor([1,1,1,1,1])

And now - let's send our tensors to Bob!!

In [0]:
x_ptr = x.send(bob)
y_ptr = y.send(bob)

In [9]:
x_ptr

(Wrapper)>[PointerTensor | me:23104891255 -> bob:1552115775]

BOOM! Now Bob has two tensors! Don't believe me? Have a look for yourself!

In [10]:
bob._objects

{1552115775: tensor([1, 2, 3, 4, 5]), 95232937004: tensor([1, 1, 1, 1, 1])}

In [0]:
z = x_ptr + x_ptr

In [12]:
z

(Wrapper)>[PointerTensor | me:10173603046 -> bob:42854278748]

In [13]:
bob._objects

{1552115775: tensor([1, 2, 3, 4, 5]),
 42854278748: tensor([ 2,  4,  6,  8, 10]),
 95232937004: tensor([1, 1, 1, 1, 1])}

Now notice something. When we called `x.send(bob)` it returned a new object that we called `x_ptr`. This is our first *pointer* to a tensor. Pointers to tensors do NOT actually hold data themselves. Instead, they simply contain metadata about a tensor (with data) stored on another machine. The purpose of these tensors is to give us an intuitive API to tell the other machine to compute functions using this tensor. Let's take a look at the metadata that pointers contain.

In [14]:
x_ptr

(Wrapper)>[PointerTensor | me:23104891255 -> bob:1552115775]

Check out that metadata!

There are two main attributes specific to pointers:

- `x_ptr.location : bob`, the location, a reference to the location that the pointer is pointing to
- `x_ptr.id_at_location : <random integer>`, the id where the tensor is stored at location

They are printed in the format `<id_at_location>@<location>`

There are also other more generic attributes:
- `x_ptr.id : <random integer>`, the id of oui pointer tensor, it was allocated randomly
- `x_ptr.owner : me`, the worker which owns the pointer tensor, here it's the local worker, named "me"


In [15]:
x_ptr.location

<VirtualWorker id:bob #tensors:3>

In [16]:
bob

<VirtualWorker id:bob #tensors:3>

In [17]:
bob == x_ptr.location

True

In [18]:
x_ptr.id_at_location

1552115775

In [19]:
x_ptr.owner

<VirtualWorker id:me #tensors:0>

You may wonder why the local worker which owns the pointer is also a VirtualWorker, although we didn't create it.
Fun fact, just like we had a VirtualWorker object for Bob, we (by default) always have one for us as well. This worker is automatically created when we called `hook = sy.TorchHook()` and so you don't usually have to create it yourself.

In [20]:
me = sy.local_worker
me

<VirtualWorker id:me #tensors:0>

In [21]:
me == x_ptr.owner

True

And finally, just like we can call .send() on a tensor, we can call .get() on a pointer to a tensor to get it back!!!

In [22]:
x_ptr

(Wrapper)>[PointerTensor | me:23104891255 -> bob:1552115775]

In [23]:
x_ptr.get()

tensor([1, 2, 3, 4, 5])

In [24]:
y_ptr

(Wrapper)>[PointerTensor | me:96004067886 -> bob:95232937004]

In [25]:
y_ptr.get()

tensor([1, 1, 1, 1, 1])

In [26]:
z.get()

tensor([ 2,  4,  6,  8, 10])

In [27]:
bob._objects

{}

And as you can see... Bob no longer has the tensors anymore!!! They've moved back to our machine!

# Section 1.2 - Using Tensor Pointers

So, sending and receiving tensors from Bob is great, but this is hardly Deep Learning! We want to be able to perform tensor _operations_ on remote tensors. Fortunately, tensor pointers make this quite easy! You can just use pointers like you would normal tensors!

In [0]:
x = torch.tensor([1,2,3,4,5]).send(bob)
y = torch.tensor([1,1,1,1,1]).send(bob)

In [0]:
z = x + y

In [30]:
z

(Wrapper)>[PointerTensor | me:56957293608 -> bob:73305221047]

And voilà! 

Behind the scenes, something very powerful happened. Instead of x and y computing an addition locally, a command was serialized and sent to Bob, who performed the computation, created a tensor z, and then returned the pointer to z back to us!

If we call .get() on the pointer, we will then receive the result back to our machine!

In [31]:
z.get()

tensor([2, 3, 4, 5, 6])

### Torch Functions

This API has been extended to all of Torch's operations!!!

In [32]:
x

(Wrapper)>[PointerTensor | me:91623748794 -> bob:63502895834]

In [33]:
y

(Wrapper)>[PointerTensor | me:52984783831 -> bob:9066397821]

In [34]:
z = torch.add(x,y)
z

(Wrapper)>[PointerTensor | me:99300551181 -> bob:13149735288]

In [35]:
z.get()

tensor([2, 3, 4, 5, 6])

### Variables (including backpropagation!)

In [0]:
x = torch.tensor([1,2,3,4,5.], requires_grad=True).send(bob)
y = torch.tensor([1,1,1,1,1.], requires_grad=True).send(bob)

In [0]:
z = (x + y).sum()

In [38]:
z.backward()

(Wrapper)>[PointerTensor | me:26231794711 -> bob:26971071667]

In [0]:
x = x.get()

In [40]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [41]:
x.grad

tensor([1., 1., 1., 1., 1.])

So as you can see, the API is really quite flexible and capable of performing nearly any operation you would normally perform in Torch on *remote data*. This lays the groundwork for our more advanced privacy preserving protocols such as Federated Learning, Secure Multi-Party Computation, and Differential Privacy !

# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Star PySyft on GitHub

The easiest way to help our community is just by starring the GitHub repos! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! At any time you can go to PySyft GitHub Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for GitHub issues marked "good first issue".

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)